In [2]:
import os

from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')



In [3]:
from langchain_openai import ChatOpenAI #랭체인 : LLM을 사용하여 애플리케이션 생성을 단순화 하도록 설계된 프레임 워크. 랭체인에서 ChatOpenAI를 가져온다.
from langchain_core.messages import HumanMessage # 기본 추상화 메세지

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

In [4]:
from langchain.document_loaders import PyPDFLoader # PDF문서 읽어주는 Pypdfloader 설치

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("C:\\Users\\owner\\Desktop\\sparta\\본캠프\\AI\\Mission2\\CH3 Mission\\인공지능산업최신동향_2024년11월호.pdf")

# 페이지 별 문서 로드
docs = loader.load()


In [5]:
from langchain.text_splitter import CharacterTextSplitter # 주어진 텍스트를 문자 단위로 분할하는 데 사용함

text_splitter = CharacterTextSplitter(
    chunk_size=100, #각 청크의 최대 길이 최대 100자까지의 텍스트가 하나의 청크에 포함된다.
    chunk_overlap=10, #인접한 청크 사이에 중복으로 포함될 문자의 수, 각 청크들은 연결부분에서 10자까지 중복가능
    length_function=len, #청크의 길이를 계산하는 함수, len 함수는 문자열의 길이를 기반으로 청크 길이를 계산함
    is_separator_regex=False, # False로 설정해서, 구분자로 정규식을 사용하지 않음.
)

splits = text_splitter.split_documents(docs)

In [6]:
from langchain_openai import OpenAIEmbeddings #OpenAI의 API를 활용하여, 각 문서를 대응하는 임베딩 벡터로 변환

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [7]:
from langchain_community.vectorstores import FAISS # FAISS = 대용량의 데이터 간의 유사도를 빠르게 계산해주는 유사도 검색 라이브러리


vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings) #from_documents = 문서 리스트와 임베딩 함수를 사용하여 FAISS 벡터 저장소를 생성

In [8]:
from langchain.vectorstores.base import VectorStore

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4}) 

In [9]:
from langchain_core.prompts import ChatPromptTemplate #ChatPromptTemplate = 대화형 상황에서 여러 메시지 입력을 기반으로 단일 메시지 응답을 생성하는데 사용하는 라이브러리
from langchain_core.runnables import RunnablePassthrough #RunnablePassthrough = 데이터를 전달하는 역할

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([  #ChatPromptTemplate.from_messages = 메서드를 사용하여 메시지 리스트로부터 ChatPromptTemplate 인스턴스를 생성하는 방식은 대화형 프롬프트를 생성
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

from langchain.chains import LLMChain #LLMChain = 프롬프트 템플릿을 LLM을 합쳐서 컴포넌트화 한 것, 입력값으로 문자열을 넣으면 프롬프트 템플릿에 의해서 프롬프트가 자동으로 완성되고, LLM 모델을 호출하여 텍스트 출력을 내주는 기능
# 컴포넌트화 = 재사용이 가능한 각각의 독립된 모듈

class SimplePassThrough:
    def invoke(self, inputs, **kwargs): #invoke = Retriever의 주요 진입점으로, 관련 문서를 검색하는 데 사용 /kwargs = Retriever에 전달할 추가 인자
        return inputs #inputs = 검색 쿼리 문자열 

class ContextToPrompt:
    def __init__(self, prompt_template):
        self.prompt_template = prompt_template
    
    def invoke(self, inputs): 
        # 문서 내용을 텍스트로 변환
        if isinstance(inputs, list): # isinstance = 타입확인
            context_text = "\n".join([doc.page_content for doc in inputs]) #.join = 매개변수로 들어온 리스트에 있는 요소 하나하나를 합쳐서 하나의 문자열로 바꾸어 반환
        else:
            context_text = inputs
        
        # 프롬프트 템플릿에 적용
        formatted_prompt = self.prompt_template.format_messages( #format_messages = 사용자의 입력을 프롬프트에 동적으로 삽입하여, 최종적으로 대화형 상황을 반영한 메시지 리스트를 생성
            context=context_text,
            question=inputs.get("question", "")
        )
        return formatted_prompt

# Retriever를 invoke() 메서드로 래핑하는 클래스 정의
class RetrieverWrapper:
    def __init__(self, retriever):
        self.retriever = retriever

    def invoke(self, inputs):
        if isinstance(inputs, dict):
            query = inputs.get("question", "")
        else:
            query = inputs
        # 검색 수행
        response_docs = self.retriever.get_relevant_documents(query)
        return response_docs

llm_chain = LLMChain(llm=model, prompt=contextual_prompt)

# RAG 체인 설정
rag_chain_debug = {
    "context": RetrieverWrapper(retriever),
    "prompt": ContextToPrompt(contextual_prompt),
    "llm": model
}


C:\Users\owner\AppData\Local\Temp\ipykernel_112252\89854149.py:49: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=model, prompt=contextual_prompt)


In [12]:
# 챗봇 구동
while True:
    print("========================")
    query = input("질문을 입력하세요 : ")
    
    # 1. Retriever로 관련 문서 검색
    response_docs = rag_chain_debug["context"].invoke({"question": query})
    
    # # 2. 문서를 프롬프트로 변환
    prompt_messages = rag_chain_debug["prompt"].invoke({
        "context": response_docs,
        "question": query
    })
    
    # # 3. LLM으로 응답 생성
    response = rag_chain_debug["llm"].invoke(prompt_messages)
    
    print("\n답변:")
    print(response.content)


답변:
인공지능 산업 동향은 다음과 같습니다:

1. **정책/법제**: 
   - 미국 민권위원회가 얼굴인식 기술 사용에 따른 민권 영향을 분석하고, 백악관 예산관리국은 책임 있는 AI 조달을 위한 지침을 발표했습니다. 
   - 유로폴은 법 집행에서 AI의 이점과 과제를 다룬 보고서를 발간하였고, OECD는 공공 부문의 AI 도입을 위한 G7 툴킷을 발표했습니다. 
   - 세계경제포럼은 생성AI 시대의 거버넌스 프레임워크를 제시했습니다.

2. **기업/산업**: 
   - 벤처 투자에서 AI 스타트업에 대한 집중이 증가하고 있으며, 메타는 동영상 생성AI 도구와 멀티모달 AI 모델을 공개했습니다. 
   - 앨런AI 연구소는 성능이 뛰어난 오픈소스 LLM '몰모'를 공개했으며, 카카오는 새로운 AI 서비스 '카나나'를 선보였습니다.

3. **기술/연구**: 
   - 2024년 노벨 물리학상과 화학상 수상자가 AI 관련 연구자들입니다. 
   - 구글 딥마인드는 반도체 칩 레이아웃을 설계하는 AI 모델 '알파칩'을 발표했습니다.

4. **인력/교육**: 
   - 가트너는 AI 도입으로 인해 엔지니어링 인력의 80%가 역량 향상이 필요하다고 예측했습니다. 
   - AI 전문가의 73%는 2025년 중 이직을 고려하고 있으며, 생성AI가 인간 근로자를 대체할 가능성은 낮다고 분석되었습니다.

전체적으로 AI 기술의 발전과 정책적 대응이 활발하며, 기업들은 AI 통합과 인력 개발에 집중하고 있습니다.

답변:
인공지능(AI)은 컴퓨터 시스템이 인간과 유사한 방식으로 작업을 수행할 수 있도록 하는 기술입니다. AI는 데이터 분석, 문제 해결, 학습, 언어 이해, 이미지 인식 등 다양한 분야에서 활용됩니다. 최근 AI는 생성AI와 같은 새로운 형태로 발전하고 있으며, 이는 텍스트, 이미지, 비디오 등을 생성하는 능력을 갖추고 있습니다.

AI의 발전은 많은 산업 분야에 영향을 미치고 있으며, 특히 기업과 정부는 AI 기술을 통해 효율성을 높이고 새로운 

KeyboardInterrupt: Interrupted by user